In [89]:
import re
import bisect
bisect.bisect_left([10, 30, 40],  41)

re.split('[^A-Za-z]+', 'ala ma kota')


['ala', 'ma', 'kota']

In [29]:
rdd_aow = sc.textFile('data/artwar.1b.txt')

In [42]:
import re
def is_heading(s):
    return re.match(r'^[IVX]+\..*', s) or s.startswith("THE END")

chapters = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): is_heading(s)).collect()
    
display(chapters)

[(u'I. Laying Plans', 15),
 (u'II. Waging War', 106),
 (u'III. Attack by Stratagem', 184),
 (u'IV. Tactical Dispositions', 266),
 (u'V. Energy', 337),
 (u'VI. Weak Points and Strong', 426),
 (u'VII. Maneuvering', 562),
 (u'VIII. Variation in Tactics', 692),
 (u'IX. The Army on the March', 748),
 (u'X. Terrain', 908),
 (u'XI. The Nine Situations', 1033),
 (u'XII. The Attack by Fire', 1287),
 (u'XIII. The Use of Spies', 1365),
 (u'THE END', 1467)]

In [55]:
chapter_bounds = sc.broadcast([ i for (title,i) in chapters])

rdd_tmp = rdd_aow.zipWithIndex() \
    .filter(lambda (s,i): s and not is_heading(s)) \
    .map(lambda (s,i): (bisect.bisect_left(chapter_bounds.value, i),(s,i))) 
    


In [95]:
def split_to_paragraphs(lines):
    paragraphs = []
    current = []
    for line, i in lines:
        if (re.match(r'^[0-9]+\.',line)):
            if current:
                paragraphs.append(" ".join(current))
            current = [line]
        elif current:
            current.append(line)
    if current:
        paragraphs.append("".join(current))
    return paragraphs
    


rdd_tmp.groupByKey() \
    .map(lambda (k,it): (k, split_to_paragraphs(it))) \
    .flatMap(lambda (k,p):( ((k,i), t) for i,t in enumerate(p))) \
    .flatMap(lambda (cp, text): ( (token, cp) for token in re.split('[^a-z\-\']+', text.lower()))) \
    .groupByKey() \
    .map(lambda (token, it): (token, list(it))) \
    .filter(lambda (token, ref): len(ref) <2 ) \
    .sortByKey() \
    .collect()

[(u'--let', [(1, 12)]),
 (u'abandoned', [(10, 3)]),
 (u'abroad', [(13, 0)]),
 (u'absolutely', [(6, 9)]),
 (u'access', [(9, 19)]),
 (u'accession', [(12, 12)]),
 (u'accommodate', [(11, 66)]),
 (u'accommodating', [(11, 59)]),
 (u'accompany', [(8, 3)]),
 (u'accomplish', [(11, 61)]),
 (u'accomplishing', [(8, 7)]),
 (u'achieved', [(6, 20)]),
 (u'acquire', [(13, 21)]),
 (u'acrid', [(5, 8)]),
 (u'activity', [(6, 22)]),
 (u'actual', [(2, 1)]),
 (u'adaptation', [(3, 14)]),
 (u'adefeat', [(3, 17)]),
 (u'adheres', [(4, 15)]),
 (u'administers', [(3, 13)]),
 (u'advances', [(10, 23)]),
 (u'advantageous', [(7, 4)]),
 (u'advent', [(10, 7)]),
 (u'advisable', [(10, 6)]),
 (u'affect', [(8, 11)]),
 (u'afflicted', [(2, 11)]),
 (u'afoot', [(9, 32)]),
 (u'aggressive', [(11, 40)]),
 (u'ahead', [(12, 15)]),
 (u'aides-de-camp', [(13, 19)]),
 (u'aim', [(13, 24)]),
 (u'alike', [(11, 22)]),
 (u'alliance', [(11, 51)]),
 (u'allotments', [(7, 19)]),
 (u'allow', [(6, 1)]),
 (u'allowing', [(13, 11)]),
 (u'allows', [(10,

In [98]:
rdd_tmp.groupByKey() \
    .map(lambda (k,it): (k, split_to_paragraphs(it))) \
    .flatMap(lambda (k,p):( ((k,i), t) for i,t in enumerate(p))) \
    .flatMap(lambda (cp, text): ( (token, cp) for token in re.split('[^a-z\-\']+', text.lower()))) \
    .groupByKey() \
    .map(lambda (token, it): (token, list(it))) \
    .filter(lambda (token, ref): len(ref) <2 ) \
    .map(lambda (token, ref):  ( ref[0][0], token )) \
    .groupByKey() \
    .map(lambda (ch, it): (ch, " ".join(it))) \
    .sortByKey() \
    .collect()

[(1,
  u'islikely based cold trained irritate foresee neglected dismissed strictness tovictory subdivisions arrogant retained evade comprises marshaling makesbut feign heads histemple devices regardless maintenance seek imbued inquiry graduations rest --let rules obtaining signifies governed avail importance heat profit forecast choleric punishment highly sincerely comparison ere generals vital calculationat fought attention pretend stands constancy undismayed rigorously inactive reward loses understood virtues basis helpful heading factors'),
 (2,
  u"per forage contributions picul poverty dissipated campaign draught-oxen augment needs levy roused including dulled arrows provender armor campaigns protracted store cleverness haste breast-plates consequences delays cartload four-tenths peasantry bare profitable nationshall swift exhaustion associated chieftains evils proximity kill sums contributing actual extremity supply-wagons protective wagons prolonged cost expenses helmets avert i